In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import glob
import pandas as pd

In [143]:
exp_folders = glob.glob('experiments/*/')
params = {}
results = {}
for exp_folder in exp_folders:
    exp_name = exp_folder.split(os.sep)[1]
    if not os.path.exists(os.path.join(exp_folder, 'params.p')):
        continue
    with open(os.path.join(exp_folder, 'params.p'), 'rb') as params_file:
        params[exp_name] = pickle.load(params_file)
    with open(os.path.join(exp_folder, 'results.p'), 'rb') as results_file:
        results[exp_name] = pickle.load(results_file)

result_df = pd.DataFrame()
params_df = pd.DataFrame()
for exp, result in results.items():
    mean_result = pd.DataFrame.from_dict(result).mean()
    mean_result.name=exp
    result_df = result_df.append(mean_result)
result_df = pd.concat([result_df, pd.DataFrame.from_dict(params).T], axis=1)
result_df[['average_precision', 'ndcg_at_10', 'n_steps']].sort_values('n_steps')

,average_precision,ndcg_at_10,n_steps
pvdm_original_articles_1_300_300_cosine,0.203961,0.228228,1031
pvdm_original_articles_5_300_300_cosine,0.926748,0.932935,5155


In [4]:
result_df

,average_precision,ndcg_at_10,algorithm,architecture,batch_size,concat,data,dist_measure,document_size,emb_size_d,...,iterations,learning_rate,loss_type,n_neg_samples,n_steps,nsteps,optimize,prior_sample_size,vocabulary_size,window_size
doc2vec-gensim_aminer_org_v1_5_300_300_cosine,0.000702,0.001029,doc2vec-gensim,pvdm,128,True,aminer_org_v1,cosine,111000,300,...,5,1,sampled_softmax_loss,64,555000,NaN,Adagrad,NaN,50000,8
doc2vec-gensim_original_articles_100001_300_300_cosine,0.677843,0.687516,doc2vec-gensim,pvdm,128,True,original_articles,cosine,1031,300,...,NaN,1,sampled_softmax_loss,64,100001,100001,Adagrad,NaN,50000,8
doc2vec-gensim_original_articles_5_100_100_cosine,0.675223,0.680724,doc2vec-gensim,pvdm,128,True,original_articles,cosine,1031,100,...,5,1,sampled_softmax_loss,64,5155,NaN,Adagrad,NaN,50000,8
doc2vec-gensim_original_articles_5_300_300_cosine,0.673040,0.682589,doc2vec-gensim,pvdm,128,True,original_articles,cosine,1031,300,...,5,1,sampled_softmax_loss,64,5155,NaN,Adagrad,NaN,50000,8
pvdm_original_articles_10000_300_300_cosine,0.649623,0.670708,pvdm,pvdm,128,True,original_articles,cosine,1031,300,...,NaN,1,sampled_softmax_loss,64,10000,10000,Adagrad,10,50000,8
pvdm_original_articles_1000_300_300_cosine,0.674012,0.685618,pvdm,pvdm,128,True,original_articles,cosine,1031,300,...,NaN,1,sampled_softmax_loss,64,1000,1000,Adagrad,10,50000,8
pvdm_original_articles_100_300_300_cosine,0.679642,0.689899,pvdm,pvdm,128,True,original_articles,cosine,1031,300,...,NaN,1,sampled_softmax_loss,64,100,100,Adagrad,10,50000,8
pvdm_original_articles_50000_300_300_cosine,0.639258,0.665104,pvdm,pvdm,128,True,original_articles,cosine,1031,300,...,NaN,1,sampled_softmax_loss,64,50000,50000,Adagrad,10,50000,8


In [36]:
import pickle
raw_data_folder = 'aminer_org_v1'
rel_labels_fname = 'relevance_labels_' + raw_data_folder + '.p'
with open(rel_labels_fname, 'rb') as rel_lab_file:
    _, _, _, tokenized, _, sorted_bm25_indices = pickle.load(rel_lab_file)

In [37]:
import numpy as np

In [43]:
import nltk
import gensim
output_fname='rel_labels.p'
folder='original_articles'
source_dict = {}  # maps article filename to source
docs = []  # list with documents
doc_names = []  # doc names with same index as docs

data_path = os.path.join(os.getcwd(), 'raw_data.tmp', folder)
for subdir, dirs, files in os.walk(data_path):
    files = [fi for fi in files if fi.endswith(".txt")]
    for file in files:
        path = os.path.join(subdir, file)
        folder_name = subdir.split(os.path.sep)[-1]
        fname = file[:-4]
        source_dict[fname] = subdir.split(os.path.sep)[-1]
        with open(path, 'r', encoding='utf8') as f:
            docs.append(f.read())
        doc_names.append(fname)

tokenized = []
for doc in docs:
    tokens = [word for sent in nltk.sent_tokenize(doc) for word in nltk.word_tokenize(sent)]
    tokenized.append(tokens)

print(len(docs))
print("Computing BM25...")
bm25 = gensim.summarization.bm25.BM25(tokenized)
print("Done computing bm25, compute average IDF...")
average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())
print("Done computing average IDF.")
bm25_scores = []
sorted_bm25_indices = []
len_tokenized = len(tokenized)

1031
Computing BM25...
Done computing bm25, compute average IDF...
Done computing average IDF.


In [55]:
doc = tokenized[500]
print(' '.join(doc))
temp_bm25_scores = bm25.get_scores(doc, average_idf)
temp_bm25_scores = temp_bm25_scores
sorted_indices = sorted(range(len(temp_bm25_scores)), key=lambda x: temp_bm25_scores[x], reverse=True)
# print(sorted_indices)

Article 517 Definition of eligible capital By 31 December 2014 the Commission shall review and report on the appropriateness of the definition of eligible capital being applied for the purposes of Title III of Part Two and Part Four and shall submit that report to the European Parliament and the Council , and , if appropriate , a legislative proposal .
867


In [101]:
import gensim
from gensim import corpora
import math

class BM25 :
    def __init__(self, fn_docs) :
        self.dictionary = corpora.Dictionary()
        self.doc_names = []
        self.source_dict = {}
        self.DF = {}
        self.DocTF = []
        self.DocIDF = {}
        self.N = 0
        self.DocAvgLen = 0
        self.fn_docs = fn_docs
        self.DocLen = []
        self.buildDictionary()
        self.TFIDF_Generator()

    def buildDictionary(self) :
        raw_data = []
#         for line in file(self.fn_docs) :
#             raw_data.append(line.strip().split(self.delimiter))
        data_path = os.path.join(os.getcwd(), 'raw_data.tmp', self.fn_docs)
        for subdir, dirs, files in os.walk(data_path):
            files = [fi for fi in files if fi.endswith(".txt")]
            for file in files:
                path = os.path.join(subdir, file)
                folder_name = subdir.split(os.path.sep)[-1]
                fname = file[:-4]
                self.source_dict[fname] = subdir.split(os.path.sep)[-1]
                with open(path, 'r', encoding='utf8') as f:
                    raw_data.append([word for sent in nltk.sent_tokenize(f.read()) for word in nltk.word_tokenize(sent)])
                self.doc_names.append(fname)
        self.dictionary.add_documents(raw_data)

    def TFIDF_Generator(self, base=math.e) :
        docTotalLen = 0
        data_path = os.path.join(os.getcwd(), 'raw_data.tmp', self.fn_docs)
        for subdir, dirs, files in os.walk(data_path):
            files = [fi for fi in files if fi.endswith(".txt")]
            for file in files:
                path = os.path.join(subdir, file)
                with open(path, 'r', encoding='utf8') as f:
                    doc = [word for sent in nltk.sent_tokenize(f.read()) for word in nltk.word_tokenize(sent)]
                docTotalLen += len(doc)
                self.DocLen.append(len(doc))
                bow = dict([(term, freq*1.0/len(doc)) for term, freq in self.dictionary.doc2bow(doc)])
                for term, tf in bow.items() :
                    if term not in self.DF :
                        self.DF[term] = 0
                    self.DF[term] += 1
                self.DocTF.append(bow)
                self.N = self.N + 1
        for term in self.DF:
            self.DocIDF[term] = math.log((self.N - self.DF[term] +0.5) / (self.DF[term] + 0.5), base)
        self.DocAvgLen = docTotalLen / self.N

    def BM25Score(self, Query=[], k1=1.5, b=0.75) :
        query_bow = self.dictionary.doc2bow(Query)
        scores = []
        for idx, doc in enumerate(self.DocTF) :
            commonTerms = set(dict(query_bow).keys()) & set(doc.keys())
            tmp_score = []
            doc_terms_len = self.DocLen[idx]
            for term in commonTerms :
                upper = (doc[term] * (k1+1))
                below = ((doc[term]) + k1*(1 - b + b*doc_terms_len/self.DocAvgLen))
                tmp_score.append(self.DocIDF[term] * upper / below)
            scores.append(sum(tmp_score))
        return scores

    def TFIDF(self) :
        tfidf = []
        for doc in self.DocTF :
            doc_tfidf  = [(term, tf*self.DocIDF[term]) for term, tf in doc.items()]
            doc_tfidf.sort()
            tfidf.append(doc_tfidf)
        return tfidf

    def Items(self) :
        # Return a list [(term_idx, term_desc),]
        items = self.dictionary.items()
        items.sort()
        return items

In [134]:
import nltk
import gensim
output_fname='rel_labels.p'
folder='aminer_org_v1'
source_dict = {}  # maps article filename to source
docs = []  # list with documents
doc_names = []  # doc names with same index as docs

data_path = os.path.join(os.getcwd(), 'raw_data.tmp', folder)
for subdir, dirs, files in os.walk(data_path):
    files = [fi for fi in files if fi.endswith(".txt")]
    for file in files:
        path = os.path.join(subdir, file)
        folder_name = subdir.split(os.path.sep)[-1]
        fname = file[:-4]
        source_dict[fname] = subdir.split(os.path.sep)[-1]
        with open(path, 'r', encoding='utf8') as f:
            docs.append(f.read())
        doc_names.append(fname)

tokenized = []
for doc in docs:
    tokens = [word for sent in nltk.sent_tokenize(doc) for word in nltk.word_tokenize(sent)]
    tokenized.append(tokens)

In [133]:
fn_docs = 'aminer_org_v1'
bm25 = BM25(fn_docs)

In [138]:
def inspect(index):
    print(index)
    Query = tokenized[index]
    scores = bm25.BM25Score(Query)
    sorted_indices = sorted(range(len(scores)), key=lambda x: scores[x], reverse=True)
    print(sorted_indices[:10])
    print('\nOriginal text:\n')
    print(' '.join(tokenized[index][:300]))
    print('\nTop 5:\n')
    for i in range(5):
        print(' '.join(tokenized[sorted_indices[i]]),'\n--------------------\n')

index = np.random.randint(len(tokenized))
inspect(index)

91283
[91283, 6339, 72175, 33383, 57814, 109712, 19367, 94624, 35338, 22166]

Original text:

Dynamically managing the communication-parallelism trade-off in future clustered processors Clustered microarchitectures are an attractive alternative to large monolithic superscalar designs due to their potential for higher clock rates in the face of increasingly wire-delay-constrained process technologies . As increasing transistor counts allow an increase in the number of clusters , thereby allowing more aggressive use of instruction-level parallelism ( ILP ) , the inter-cluster communication increases as data values get spread across a wider area . As a result of the emergence of this trade-off between communication and parallelism , a subset of the total on-chip clusters is optimal for performance . To match the hardware to the application 's needs , we use a robust algorithm to dynamically tune the clustered architecture . The algorithm , which is based on program metrics gathered at per

In [68]:
fn_docs = 'original_articles'


raw_data = []
data_path = os.path.join(os.getcwd(), 'raw_data.tmp', fn_docs)
print(data_path)
doc_names=[]
dictionary=corpora.Dictionary()
for subdir, dirs, files in os.walk(data_path):
    files = [fi for fi in files if fi.endswith(".txt")]
    for file in files:
        path = os.path.join(subdir, file)
        folder_name = subdir.split(os.path.sep)[-1]
        fname = file[:-4]
        source_dict[fname] = subdir.split(os.path.sep)[-1]
        with open(path, 'r', encoding='utf8') as f:
            raw_data.append([word for sent in nltk.sent_tokenize(f.read()) for word in nltk.word_tokenize(sent)])
        doc_names.append(fname)
dictionary.add_documents(raw_data)

/Users/alexandervansomeren/Documents/Studie/Msc_AI/Thesis/regulatory tracker/doc2vec_pipeline/raw_data.tmp/original_articles


In [77]:
dictionary.doc2bow()

TypeError: doc2bow() missing 1 required positional argument: 'document'